In [1]:
# ROBUST MODEL TESTING SCRIPT - Uses all saved artifacts with error handling
import os
import json
import joblib
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.model_selection import TimeSeriesSplit, GroupKFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

print("=== PROFESSIONAL PM2.5 MODEL TESTING SUITE ===")
print("Loading all artifacts and performing comprehensive model validation...")

# Configuration
SEED = 42
USE_LOG_TARGET = True

def inv_if_log(a):
    """Inverse log1p transform if model was trained on log scale"""
    return np.expm1(a) if USE_LOG_TARGET else a

# Step 1: Verify and load all required artifacts
print("\n1. ARTIFACT VERIFICATION")
required_files = {
    "model": "model_xgb_prod_phase6.joblib",
    "features": "features_phase2_locked.json", 
    "metadata": "model_metadata_phase6.json",
    "dataset": "dataset_phase2_leakage_safe.parquet"
}

# Check if files exist, with fallbacks
for name, filename in required_files.items():
    if not os.path.exists(filename):
        if name == "model":
            # Try alternative model names
            alternatives = ["model_xgb_phase4_final.joblib", "pm25_model_production.joblib"]
            for alt in alternatives:
                if os.path.exists(alt):
                    required_files["model"] = alt
                    print(f"✓ Using alternative model: {alt}")
                    break
            else:
                raise FileNotFoundError(f"No model file found. Tried: {filename}, {alternatives}")
        elif name == "dataset":
            # Try Phase 1 dataset and rebuild features
            alt = "dataset_feature1_phase1_clean.parquet"
            if os.path.exists(alt):
                required_files["dataset"] = alt
                print(f"✓ Using Phase 1 dataset: {alt} (will rebuild AR features)")
            else:
                raise FileNotFoundError(f"No dataset found. Tried: {filename}, {alt}")
        else:
            raise FileNotFoundError(f"Required file not found: {filename}")
    else:
        print(f"✓ Found: {filename}")

# Step 2: Load model and metadata
print("\n2. MODEL LOADING")
try:
    model = joblib.load(required_files["model"])
    print(f"✓ Model loaded from: {required_files['model']}")
    print(f"  Model type: {type(model).__name__}")
except Exception as e:
    raise RuntimeError(f"Failed to load model: {e}")

# Load feature list
try:
    with open(required_files["features"], "r") as f:
        feature_data = json.load(f)
    features = feature_data["features"] if isinstance(feature_data, dict) else feature_data
    print(f"✓ Feature list loaded: {len(features)} features")
except Exception as e:
    raise RuntimeError(f"Failed to load features: {e}")

# Load metadata if available
metadata = {}
try:
    with open(required_files["metadata"], "r") as f:
        metadata = json.load(f)
    print(f"✓ Metadata loaded")
except Exception as e:
    print(f"⚠ Metadata not available: {e}")

# Step 3: Load and prepare dataset
print("\n3. DATASET PREPARATION")
try:
    df = pd.read_parquet(required_files["dataset"])
    print(f"✓ Dataset loaded: {df.shape}")
    
    # If using Phase 1 dataset, rebuild AR features
    if "phase1" in required_files["dataset"]:
        print("  Rebuilding leakage-safe AR features...")
        
        # Ensure numeric station_quality
        if "station_quality" in df.columns and pd.api.types.is_categorical_dtype(df["station_quality"]):
            df["station_quality"] = df["station_quality"].cat.codes
        
        # Rebuild AR features
        df = df.sort_values(["station_id", "timestamp_utc"]).copy()
        g = df.groupby("station_id")["PM2.5"]
        
        # Lags
        for h in [1,3,6,12,24,48,72]:
            col = f"pm25_lag{h}"
            if col not in df.columns:
                df[col] = g.shift(h)
        
        # Past-only rollings
        s = g.shift(1)  # Exclude current t
        for w in [3,6,12,24]:
            mean_col = f"PM25_past_rolling{w}_mean"
            std_col = f"PM25_past_rolling{w}_std"
            if mean_col not in df.columns:
                df[mean_col] = s.rolling(w, min_periods=max(1,w//2)).mean()
            if std_col not in df.columns:
                df[std_col] = s.rolling(w, min_periods=max(2,w//2)).std()
        
        # Interactions (only if base features exist)
        if all(c in df.columns for c in ["pm25_lag3", "stability"]):
            df["lag3_x_stability"] = df["pm25_lag3"] * df["stability"]
        
        if "tp" in df.columns and "tp_rolling24" not in df.columns:
            s_tp = df.groupby("station_id")["tp"].shift(1)
            df["tp_rolling24"] = s_tp.rolling(24, min_periods=12).mean()
        
        if all(c in df.columns for c in ["pm25_lag3", "tp_rolling24"]):
            df["lag3_x_tp24"] = df["pm25_lag3"] * df["tp_rolling24"]
        
        print(f"  AR features rebuilt. New shape: {df.shape}")
    
    # Create target variable
    if USE_LOG_TARGET and "PM25_log1p" not in df.columns:
        df["PM25_log1p"] = np.log1p(df["PM2.5"])
    
    # Verify features exist and filter to available ones
    available_features = [f for f in features if f in df.columns]
    missing_features = [f for f in features if f not in df.columns]
    
    if missing_features:
        print(f"⚠ Missing {len(missing_features)} features: {missing_features[:5]}...")
        print(f"  Using {len(available_features)} available features")
        features = available_features
    else:
        print(f"✓ All {len(features)} features available")
    
    # Prepare X and y
    X = df[features].copy()
    y = df["PM25_log1p"] if USE_LOG_TARGET else df["PM2.5"]
    
    # Clean any remaining issues
    for col in X.columns:
        if pd.api.types.is_categorical_dtype(X[col]):
            X[col] = X[col].cat.codes
        elif X[col].dtype == 'object':
            try:
                X[col] = pd.to_numeric(X[col])
            except:
                X[col] = pd.factorize(X[col])[0]
    
    print(f"✓ Final dataset ready: X{X.shape}, y{y.shape}")
    
except Exception as e:
    raise RuntimeError(f"Dataset preparation failed: {e}")

# Step 4: Model validation tests
print("\n4. MODEL VALIDATION TESTS")

# Test 4.1: Basic prediction test
print("\n4.1 Basic Prediction Test")
try:
    sample_X = X.head(100).fillna(X.mean())  # Handle any NaNs for testing
    sample_pred = model.predict(sample_X)
    sample_pred_orig = inv_if_log(sample_pred)
    
    print(f"✓ Prediction test passed")
    print(f"  Sample predictions: {sample_pred_orig[:5].round(2)}")
    print(f"  Prediction range: {sample_pred_orig.min():.1f} - {sample_pred_orig.max():.1f} μg/m³")
except Exception as e:
    print(f"✗ Prediction test failed: {e}")
    raise

# Test 4.2: TimeSeriesSplit Cross-Validation
print("\n4.2 TimeSeriesSplit Cross-Validation")
try:
    tss = TimeSeriesSplit(n_splits=5, gap=3)
    cv_results = []
    
    for fold, (tr, va) in enumerate(tss.split(X), 1):
        # Handle NaNs in targets
        tr_mask = ~y.iloc[tr].isna()
        va_mask = ~y.iloc[va].isna()
        
        if tr_mask.sum() == 0 or va_mask.sum() == 0:
            continue
            
        tr_idx = np.array(tr)[tr_mask]
        va_idx = np.array(va)[va_mask]
        
        X_va, y_va = X.iloc[va_idx], y.iloc[va_idx]
        
        # Predict and evaluate
        pred = model.predict(X_va)
        y_true = inv_if_log(y_va)
        y_pred = inv_if_log(pred)
        
        rmse = sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        
        cv_results.append({
            "fold": fold,
            "rmse": rmse,
            "mae": mae,
            "r2": r2,
            "n_samples": len(y_true)
        })
    
    cv_df = pd.DataFrame(cv_results)
    print("✓ TimeSeriesSplit Results:")
    print(cv_df.round(3))
    print(f"  Mean RMSE: {cv_df['rmse'].mean():.2f} ± {cv_df['rmse'].std():.2f}")
    print(f"  Mean R²: {cv_df['r2'].mean():.3f} ± {cv_df['r2'].std():.3f}")
    
except Exception as e:
    print(f"✗ TimeSeriesSplit test failed: {e}")

# Test 4.3: Leave-One-Station-Out (LOSO)
print("\n4.3 Leave-One-Station-Out Test")
try:
    if "station_id" in df.columns:
        groups = df["station_id"].values
        n_stations = len(np.unique(groups))
        gkf = GroupKFold(n_splits=min(5, n_stations))
        loso_results = []
        
        for fold, (tr, te) in enumerate(gkf.split(X, y, groups), 1):
            X_te, y_te = X.iloc[te], y.iloc[te]
            
            pred = model.predict(X_te)
            y_true = inv_if_log(y_te)
            y_pred = inv_if_log(pred)
            
            rmse = sqrt(mean_squared_error(y_true, y_pred))
            r2 = r2_score(y_true, y_pred)
            
            loso_results.append({
                "station_fold": fold,
                "rmse": rmse,
                "r2": r2,
                "n_samples": len(y_true)
            })
        
        loso_df = pd.DataFrame(loso_results)
        print("✓ LOSO Results:")
        print(loso_df.round(3))
        print(f"  Mean RMSE: {loso_df['rmse'].mean():.2f} ± {loso_df['rmse'].std():.2f}")
        print(f"  Mean R²: {loso_df['r2'].mean():.3f} ± {loso_df['r2'].std():.3f}")
    else:
        print("⚠ No station_id column found, skipping LOSO test")
        
except Exception as e:
    print(f"✗ LOSO test failed: {e}")

# Test 4.4: Feature importance verification
print("\n4.4 Feature Importance Analysis")
try:
    if hasattr(model, 'feature_importances_'):
        importances = pd.Series(model.feature_importances_, index=features)
        top_features = importances.nlargest(10)
        print("✓ Top 10 Feature Importances:")
        for feat, imp in top_features.items():
            print(f"  {feat}: {imp:.4f}")
    else:
        print("⚠ Model does not have feature_importances_ attribute")
except Exception as e:
    print(f"✗ Feature importance test failed: {e}")

# Step 5: Summary Report
print("\n5. FINAL VALIDATION SUMMARY")
print("="*50)

try:
    # Overall statistics
    last_pred = model.predict(X.sample(min(1000, len(X)), random_state=SEED))
    last_true = y.sample(min(1000, len(y)), random_state=SEED)
    overall_rmse = sqrt(mean_squared_error(inv_if_log(last_true), inv_if_log(last_pred)))
    
    print(f"Model Type: {type(model).__name__}")
    print(f"Features: {len(features)}")
    print(f"Training Data: {len(X):,} samples")
    print(f"Stations: {df['station_id'].nunique() if 'station_id' in df.columns else 'Unknown'}")
    print(f"Target Transform: {'log1p' if USE_LOG_TARGET else 'none'}")
    print(f"Sample RMSE: {overall_rmse:.2f} μg/m³")
    
    if 'cv_results' in locals():
        print(f"CV RMSE: {cv_df['rmse'].mean():.2f} ± {cv_df['rmse'].std():.2f}")
        print(f"CV R²: {cv_df['r2'].mean():.3f} ± {cv_df['r2'].std():.3f}")
    
    if 'loso_results' in locals():
        print(f"LOSO RMSE: {loso_df['rmse'].mean():.2f} ± {loso_df['rmse'].std():.2f}")
        print(f"Spatial Generalization: {'Good' if loso_df['rmse'].mean() < 40 else 'Moderate'}")
    
    print("\n✓ MODEL IS READY FOR DEPLOYMENT")
    
except Exception as e:
    print(f"✗ Summary generation failed: {e}")

print("\n=== TESTING COMPLETE ===")

=== PROFESSIONAL PM2.5 MODEL TESTING SUITE ===
Loading all artifacts and performing comprehensive model validation...

1. ARTIFACT VERIFICATION
✓ Found: model_xgb_prod_phase6.joblib
✓ Found: features_phase2_locked.json
✓ Found: model_metadata_phase6.json
✓ Found: dataset_phase2_leakage_safe.parquet

2. MODEL LOADING
✓ Model loaded from: model_xgb_prod_phase6.joblib
  Model type: XGBRegressor
✓ Feature list loaded: 72 features
✓ Metadata loaded

3. DATASET PREPARATION
✓ Dataset loaded: (102720, 167)
✓ All 72 features available
✓ Final dataset ready: X(102720, 72), y(102720,)

4. MODEL VALIDATION TESTS

4.1 Basic Prediction Test
✓ Prediction test passed
  Sample predictions: [ 92.26  97.2  102.77 180.54 191.54]
  Prediction range: 39.1 - 244.9 μg/m³

4.2 TimeSeriesSplit Cross-Validation
✓ TimeSeriesSplit Results:
   fold    rmse     mae     r2  n_samples
0     1  32.160  18.077  0.868      17120
1     2  33.560  18.183  0.724      17120
2     3  21.843   6.616  0.578      17120
3     4  

## Unit Test for Lag/Rolling Leakage

In [2]:
import pandas as pd
import numpy as np

def test_shift_then_roll_no_leakage():
    # Create synthetic data
    timestamps = pd.date_range("2025-01-01", periods=10, freq="H")
    df = pd.DataFrame({
        "station_id": ["S1"] * 10,
        "timestamp_utc": timestamps,
        "PM2.5": np.arange(10, 20, dtype=float)
    })

    # Build lag1 and roll3_mean incorrectly
    df["bad_roll3"] = df.groupby("station_id")["PM2.5"].rolling(3).mean().reset_index(0, drop=True)
    # Build correctly: shift before rolling
    grp = df.groupby("station_id")["PM2.5"]
    df["good_roll3"] = grp.shift(1).rolling(3).mean().reset_index(0, drop=True)

    # At index 2 (third hour), bad_roll3 includes PM2.5 at t=2 => should equal mean(10,11,12) = 11
    assert df.loc[2, "bad_roll3"] == np.mean([10,11,12])
    # good_roll3 at index 2 uses values at t=0,1 => mean(10,11) = 10.5
    assert np.isclose(df.loc[2, "good_roll3"], 10.5)
    
    # Check that no good_roll3 value equals the current PM2.5
    leaks = (df["good_roll3"] == df["PM2.5"])
    assert not leaks.any(), "Leakage detected: good_roll3 should never match current PM2.5"

## Nested CV Wrapper for Feature Selection + Modeling

In [6]:
# Full fixed script: Corr+VIF selector + nested TSS CV + XGBoost early stopping (robust)
import json
import inspect
import warnings
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import xgboost as xgb
import statsmodels.api as sm

SEED = 42

# -----------------------------
# Corr+VIF selector (unchanged logic, with safety guards)
# -----------------------------
class CorrVIFSelector(BaseEstimator, TransformerMixin):
    def __init__(self, corr_thresh=0.85, vif_thresh=10.0, protected=None):
        self.corr_thresh = corr_thresh
        self.vif_thresh = vif_thresh
        self.protected = set(protected or [])

    def fit(self, X, y=None):
        df = X.copy().astype(float, errors="ignore")

        # correlation clustering
        corr = df.corr().abs()
        to_drop = set()
        cols = list(corr.columns)
        for i in range(len(cols)):
            for j in range(i+1, len(cols)):
                a, b = cols[i], cols[j]
                if corr.loc[a, b] > self.corr_thresh:
                    # drop the one with smaller mean-correlation
                    drop = b if corr[a].mean() >= corr[b].mean() else a
                    to_drop.add(drop)
        feats = [f for f in df.columns if f not in to_drop]

        # VIF pruning with safety for small sets
        def compute_vif(df_sel):
            # return dict {col: vif}
            if df_sel.shape[1] <= 1:
                return {col: 1.0 for col in df_sel.columns}
            Xc = df_sel.fillna(df_sel.median(numeric_only=True))
            Xc = sm.add_constant(Xc)
            vifs = {}
            for i, col in enumerate(df_sel.columns):
                # safety guard: if there's no i+1 column, skip
                if i+1 >= Xc.shape[1]:
                    vifs[col] = 1.0
                    continue
                ycol = Xc.iloc[:, i+1]
                Xother = Xc.drop(Xc.columns[i+1], axis=1)
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    r2 = sm.OLS(ycol, Xother).fit().rsquared
                vifs[col] = np.inf if (1 - r2) == 0 else 1.0 / (1.0 - r2)
            return vifs

        keep = feats.copy()
        while True:
            if len(keep) <= 1:
                break
            vifs = compute_vif(df[keep])
            # pick worst VIF
            worst, val = max(vifs.items(), key=lambda kv: kv[1])
            if val <= self.vif_thresh:
                break
            if worst in self.protected:
                # try to find next non-protected offender
                offenders = [f for f, v in vifs.items() if (f not in self.protected) and (v > self.vif_thresh)]
                if offenders:
                    worst = offenders[0]
                    val = vifs[worst]
                else:
                    # nothing safe to drop
                    break
            # drop worst
            keep.remove(worst)
        self.selected_features_ = keep
        return self

    def transform(self, X):
        return X[self.selected_features_].copy()

# -----------------------------
# Robust XGBoost fit helper
# -----------------------------
class BoosterWrapper:
    """Wrap xgboost.Booster to provide .predict like sklearn estimator"""
    def __init__(self, booster):
        self.booster = booster
        self.best_iteration = getattr(booster, "best_iteration", None)
    def predict(self, Xp):
        # accept pandas DataFrame or numpy array
        return self.booster.predict(xgb.DMatrix(Xp))

def fit_xgb_with_es(params, X_tr, y_tr, X_va, y_va, early_rounds=50, verbose=False):
    """
    Fit an XGBoost model with early stopping in a way that works across versions.
    Returns either a fitted XGBRegressor or BoosterWrapper with .predict(X) semantics.
    """
    # ensure copies to avoid side-effects
    X_tr = X_tr.copy()
    X_va = X_va.copy()
    # encode categorical columns to integer codes (if any)
    for c in X_tr.select_dtypes(include=["category"]).columns:
        X_tr[c] = X_tr[c].cat.codes
    for c in X_va.select_dtypes(include=["category"]).columns:
        X_va[c] = X_va[c].cat.codes
    for c in X_tr.select_dtypes(include=["object"]).columns:
        try:
            X_tr[c] = pd.to_numeric(X_tr[c])
        except Exception:
            X_tr[c] = pd.factorize(X_tr[c])[0]
    for c in X_va.select_dtypes(include=["object"]).columns:
        try:
            X_va[c] = pd.to_numeric(X_va[c])
        except Exception:
            X_va[c] = pd.factorize(X_va[c])[0]

    # Inspect sklearn wrapper fit signature
    try:
        fit_sig = inspect.signature(XGBRegressor.fit)
        fit_params = set(fit_sig.parameters.keys())
    except Exception:
        fit_params = set()

    # create sklearn wrapper
    model = XGBRegressor(**params)

    # 1) Try callbacks API
    if "callbacks" in fit_params:
        try:
            # try import EarlyStopping from xgboost.callback if available
            try:
                from xgboost.callback import EarlyStopping
                es_cb = EarlyStopping(rounds=early_rounds, save_best=True, maximize=False)
            except Exception:
                es_cb = xgb.callback.EarlyStopping(rounds=early_rounds, save_best=True)
            model.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], verbose=verbose, callbacks=[es_cb])
            return model
        except TypeError:
            warnings.warn("callbacks path failed: falling back", RuntimeWarning)
        except Exception as e:
            warnings.warn(f"callbacks path raised {e}; falling back", RuntimeWarning)

    # 2) Try early_stopping_rounds param
    if "early_stopping_rounds" in fit_params:
        try:
            model.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], verbose=verbose, early_stopping_rounds=early_rounds)
            return model
        except TypeError:
            warnings.warn("early_stopping_rounds path failed: falling back", RuntimeWarning)
        except Exception as e:
            warnings.warn(f"early_stopping_rounds path raised {e}; falling back", RuntimeWarning)

    # 3) Fallback to xgb.train()
    xgb_params = params.copy()
    num_round = int(xgb_params.pop("n_estimators", 1000))
    # map sklearn-style keys
    if "random_state" in xgb_params:
        xgb_params["seed"] = xgb_params.pop("random_state")
    if "n_jobs" in xgb_params:
        xgb_params["nthread"] = int(xgb_params.pop("n_jobs"))
    xgb_params.setdefault("objective", "reg:squarederror")
    xgb_params.setdefault("eval_metric", "rmse")

    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dval = xgb.DMatrix(X_va, label=y_va)
    watchlist = [(dval, "validation")]
    bst = xgb.train(xgb_params, dtrain, num_boost_round=num_round, evals=watchlist,
                    early_stopping_rounds=early_rounds, verbose_eval=False)
    return BoosterWrapper(bst)

# -----------------------------
# Main experiment (nested TSS CV)
# -----------------------------
# load dataset & locked features
df = pd.read_parquet("dataset_phase2_leakage_safe.parquet")
with open("features_phase2_locked.json") as f:
    features = json.load(f)["features"]

# ensure feature presence (do not silently drop locked features)
missing = [f for f in features if f not in df.columns]
if missing:
    raise KeyError(f"Locked features missing from dataset: {missing}\n"
                   "You must run the feature-engineering pipeline that produced them.")

X = df[features].copy()
y = np.log1p(df["PM2.5"])

# coerce object/category columns to numeric safely
for c in X.columns:
    if pd.api.types.is_categorical_dtype(X[c]):
        X[c] = X[c].cat.codes
    elif X[c].dtype == object:
        try:
            X[c] = pd.to_numeric(X[c])
        except Exception:
            X[c] = pd.factorize(X[c])[0]

# example tuned params
best_params = {
    "n_estimators": 1000,
    "learning_rate": 0.05,
    "max_depth": 8,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "tree_method": "hist",
    "random_state": 42,
    "n_jobs": -1,
}

tss = TimeSeriesSplit(n_splits=5, gap=3)
results = []

for fold_i, (tr, va) in enumerate(tss.split(X), start=1):
    X_tr, X_va = X.iloc[tr], X.iloc[va]
    y_tr, y_va = y.iloc[tr], y.iloc[va]

    # 1) Fit selector on training fold only
    selector = CorrVIFSelector(protected=["PM2.5_lag3", "stability"])
    selector.fit(X_tr)
    X_tr_sel = selector.transform(X_tr)
    X_va_sel = selector.transform(X_va)

    # 2) Train using robust helper that will adapt to your XGBoost
    model = fit_xgb_with_es(best_params, X_tr_sel, y_tr, X_va_sel, y_va, early_rounds=50, verbose=False)

    # 3) Evaluate
    pred = model.predict(X_va_sel)
    rmse = np.sqrt(mean_squared_error(np.expm1(y_va), np.expm1(pred)))
    results.append(rmse)

    print(f"Fold {fold_i} RMSE: {rmse:.4f}")

print("Nested CV RMSEs:", results)
print("Mean RMSE:", np.mean(results))

Fold 1 RMSE: 37.0762
Fold 2 RMSE: 42.2544
Fold 3 RMSE: 24.6084
Fold 4 RMSE: 34.1179
Fold 5 RMSE: 29.8785
Nested CV RMSEs: [np.float64(37.076218566915074), np.float64(42.254445007820486), np.float64(24.608426791022765), np.float64(34.11793025207396), np.float64(29.878480708095157)]
Mean RMSE: 33.58710026518549
